# Rebuild FaIR 2.1

FaIR 2.0 is wonderfully simple. Possibly too simple.

Here I try to faithfully reproduce the codebase of FaIR 2.0, but with the following modifications:
- use the exact aerosol forcing relationship from Smith et al. (2021). FaIR 2.0 is trying to fit a round peg into a square hole here.
- include overlap of the major GHGs. Despite protestations from others, this is too simple in FaIR 2.0.
- prognostic equation for land use related forcing (e.g. from FaIR 1.6).
- ozone relationship from FaIR 1.6 used in AR6 (should be easy to do).

Bear in mind: would like to couple to `openscm-runner` and `mesmer`.

## First: emissions to concentrations for GHGs

Should be relatively straightforward, particularly for non-CO2 and non-CH4. FaIR 2.0 results should reproduce.

Also, RCMIP's emissions and concentrations are inconsistent. CFC11 has emissions starting in 1943, but no delta concentrations until 1945. (unless there's some kind of 3 year delay going on for stratospheric mixing, but that's only relevant for ozone).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.interpolate import interp1d

In [ ]:
# next thing is to do unit parsing!
M_ATMOS = 5.1352e18

molwt = {}
molwt["CO2"] = 44.009
molwt["C"] = 12.011
molwt["AIR"] = 28.97
molwt["CFC-11"] = 137.36

lifetime = {}
lifetime['CFC-11'] = 52

gas_list = [
    "CFC-11",
    "CO2"
]
concentration_growth_unit = {}
for gas in gas_list:
    concentration_growth_unit[gas] = 1 / (M_ATMOS / 1e18 * molwt[gas] / molwt["AIR"])
    
partition_fractions = np.array([0.2173,0.2240,0.2824,0.2763])


In [ ]:
def step_concentration_1box(
    emissions, 
    g_a_old, 
    lifetime, 
    concentration_growth_unit,
    pre_industrial_concentration=0,
    timestep=1
):
    """
    Calculates the concentrations in next timestep given emissions.
    
    Inputs
    ------
    g_a_old : float
        the sum of r_old, I think, or equal to r_old. This is the concentrations above the PI control.
    emissions: float
        emissions in timestep.
    concentration_growth_unit:
        how much atmospheric concentrations grow (e.g. in ppt) per unit (e.g. kt) emission.
    alpha_lifetime : float
        scaling factor for the default atmospheric lifetimes.
    partition_fractions : float or `np.ndarray`
        partition fractions for multi-box gases. Should be =1 if float or sum to 1 if array.
    lifetime : float or `np.ndarray`
        default atmospheric lifetime
    pre_industrial_concentration : float
        guess.
    timestep : float
        emissions timestep in years.
        
    Notes
    -----
    Emissions are given in time intervals and concentrations are also reported on the same time
        intervals: the g_a values are on time boundaries and these are averaged before being
        returned.

        
    """
    decay_rate = timestep/lifetime
    decay_factor = np.exp(-decay_rate)
    
    # Nick says: there shouldn't be a dt in the first decay rate
    # Chris says: there should, and there should be one here too. Emissions are a rate, e.g. Gt / yr
    g_a_new = emissions * 1 / decay_rate * (1 - decay_factor) * timestep + g_a_old * decay_factor
    
    # Here I have reverted to a fully forward model
    concentration_out = pre_industrial_concentration + concentration_growth_unit * (g_a_new + g_a_old) / 2
    
    return concentration_out, g_a_new

In [ ]:
# grab some emissions
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
co2_emissions = df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Emissions|CO2') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()
cfc11_emissions = df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Emissions|Montreal Gases|CFC|CFC11') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()

In [ ]:
# grab some concentrations
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
co2_concentrations_rfmip = df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Atmospheric Concentrations|CO2') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()
cfc11_concentrations_rfmip = df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Atmospheric Concentrations|Montreal Gases|CFC|CFC11') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()

In [ ]:
# test first with CFC11, inocuous (but ozone depleting... so should pass this feedback on)
cfc11_emissions

In [ ]:
concentration_growth_unit['CFC-11']  # ppt increase per kt CFC-11 emission?

In [ ]:
cfc11_concentrations = np.ones(len(cfc11_emissions)) * np.nan

g_a = 0
pre_industrical_concentration = 0
cfc11_concentrations[0] = pre_industrical_concentration  # TODO: provide pre-defined PI controls

for i in range(len(cfc11_emissions)):
    cfc11_concentrations[i], g_a = step_concentration_1box(
        cfc11_emissions[i], 
        g_a,
        lifetime['CFC-11'], #use old lifetime
        concentration_growth_unit['CFC-11'],
        timestep=1
    )

In [ ]:
# try a monthly timestep
t_new = np.arange(1750+1/24, 2501, 1/12)
f = interp1d(np.arange(1750.5, 2501), cfc11_emissions, fill_value="extrapolate")
cfc11_emissions_monthly = f(t_new)

In [ ]:
cfc11_concentrations_monthly = np.ones(len(cfc11_emissions_monthly)) * np.nan

g_a = 0
pre_industrical_concentration = 0
cfc11_concentrations_monthly[0] = pre_industrical_concentration  # TODO: provide pre-defined PI controls

for i in range(len(cfc11_concentrations_monthly)):
    cfc11_concentrations_monthly[i], g_a = step_concentration_1box(
        cfc11_emissions_monthly[i], 
        g_a,
        lifetime['CFC-11'], #use old lifetime
        concentration_growth_unit['CFC-11'],
        timestep=1/12
    )

In [ ]:
pl.plot(np.arange(1750.5, 2501), cfc11_concentrations)
pl.plot(np.arange(1750.5, 2501), cfc11_concentrations_rfmip)
pl.plot(np.arange(1750+1/24, 2501, 1/12), cfc11_concentrations_monthly)

In [ ]:
# convert to GtC from MtCO2
molwt["C"]/molwt["CO2"]/1000 * co2_emissions

In [ ]:
def step_concentration_co2(
    emissions, 
    r_old,
    g_a_old, 
    concentration_growth_unit, # could be taken from constants
    alpha_lifetime=1,
    pre_industrial_concentration=278,  # put in a defaults module
    timestep=1,
    partition_fractions = partition_fractions,  
    lifetime = np.array([1000000,394.4,36.54,4.304]),
):
    """
    Calculates the concentrations in next timestep given emissions.
    
    Inputs
    ------
    r_old : float
    
    g_a_old : float
        the sum of r_old, I think, or equal to r_old. This is the concentrations above the PI control.
    emissions: float
        emissions in timestep.
    concentration_growth_unit:
        how much atmospheric concentrations grow (e.g. in ppt) per unit (e.g. kt) emission.
    alpha_lifetime : float
        scaling factor for the default atmospheric lifetimes.
    partition_fractions : float or `np.ndarray`
        partition fractions for multi-box gases. Should be =1 if float or sum to 1 if array.
    lifetime : float or `np.ndarray`
        default atmospheric lifetime
    pre_industrial_concentration : float
        guess.
    timestep : float
        emissions timestep in years.
        
    Notes
    -----
    Emissions are given in time intervals and concentrations are also reported on the same time
        intervals: the g_a values are on time boundaries and these are averaged before being
        returned.

        
    """
    decay_rate = timestep/(alpha_lifetime * lifetime)
    decay_factor = np.exp(-decay_rate)
    
    r_new = emissions * 1 / decay_rate * (1 - decay_factor) * timestep + r_old * decay_factor
    g_a_new = np.sum(r_new)
    
    # Here I have reverted to a fully forward model
    concentration_out = pre_industrial_concentration + concentration_growth_unit * (g_a_new + g_a_old) / 2
    
    return concentration_out, r_new, g_a_new

In [ ]:
# taken from the GIR implementation in FaIR 1.6: plus r_A

def calculate_alpha(
    cumulative_emissions,
    airborne_emissions,
    temperature,
    r0,
    rC,
    rT,
    ra,
    g0,
    g1,
    iirf_max = 97.0,  # TODO: no explicit dependence on 100 year time horizon.
):
    """
    Calculate greenhouse-gas time constant scaling factor.
    
    Inputs
    ------
        cumulative_emissions : float
            GtC cumulative emissions since pre-industrial.
        airborne_emissions : float
            GtC total emissions remaining in the atmosphere.
        temperature : float
            K temperature anomaly since pre-industrial.
        r0 : float
            pre-industrial 100-year time-integrated airborne fraction.
        rc : float
            sensitivity of 100-year time-integrated airborne fraction with atmospheric carbon stock.
        rt : float
            sensitivity of 100-year time-integrated airborne fraction with temperature anomaly.
        ra : float
            sensitivity of 100-year time-integrated airborne fraction with airborne emissions. 
        g0 : float
            parameter for alpha TODO: description
        g1 : float
            parameter for alpha TODO: description
        iirf_max : float
            maximum allowable value to 100-year time-integrated airborne fraction
    Outputs
    -------
        alpha: scaling factor.
    """
    
    iirf = r0 + rc * (cumulative_emissions-airborne_emissions) + rt * temperature + ra * airborne_emissions
    iirf = (iirf>iirf_max) * iirf_max + iirf * (iirf<iirf_max)
    alpha = g0 * np.sinh(iirf / g1)
    
    return alpha

In [ ]:
co2_concentrations = np.ones(len(co2_emissions)) * np.nan

r_a = 0  # confusion with alpha parameter
airborne_emissions = 0
pre_industrical_concentration = 278.3
co2_concentrations[0] = pre_industrical_concentration  # TODO: provide pre-defined PI controls
co2_cumulative_emissions = np.cumsum(co2_emissions)
temperature = 0  # for now!
r0 = 33
rC = 0.02
rT = 2.5
rA = 0.002  # we usually turn it off

g1 = np.sum(a*tau * (1 - (1 + iirf_h/tau) * np.exp(-iirf_h/tau)))
g0 = 1/(np.sinh(np.sum(a*tau*(1 - np.exp(-iirf_h/tau)) , axis=-1)/g1))


for i in range(len(co2_emissions)):
    
    alpha = calculate_alpha(
        co2_cumulative_emissions,
        airborne_emissions,
        temperature,
        r0,
        rC,
        rT,
        ra,
        g0,
        g1,
    )
    
    co2_concentrations[i], r_a, airborne_emissions = step_concentration_co2(
        co2_emissions[i], 
        r_a,
        airborne_emissions, 
        1/2.12, # could be taken from constants
        alpha_lifetime=1,
        pre_industrial_concentration=278.3,  # put in a defaults module
        timestep=1,
    )